In [9]:
import os
import tqdm
import wandb
import warnings
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent.futures

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

from matplotlib.axes import Axes
from wandb.apis.public import Run

from typing import Union, List, Dict
from src.visualization import set_themes

set_themes() # Set custom themes for plots
pl.Config.set_tbl_rows(20) # Set Polars table display rows limit

pd.set_option('future.no_silent_downcasting', True)

In [10]:
cache_file = "wandb/summary.parquet"
config = {
    "model": "matrix_factorization",
    "ensure_available_locally": False
}
sorting_criterion = {
    "epoch/test_hitrate@50": 0.5,
    "epoch/test_ndcg@50": 0.25,
}

if cache_file is not None and os.path.exists(cache_file):
    print(f"Loading cached experiment runs from {cache_file}...")
    experiment_runs = pl.read_parquet(cache_file)
    print(f"Loaded {len(experiment_runs)} runs from cache.")
else:
    print("No cache file found. Fetching experiment runs from Weights & Biases...")
    api = wandb.Api() # Initialize Weights & Biases API, used for fetching run data

    def fetch_run_metadata(run: Run, considered_metrics: Union[str, Dict[str, float]] = "epoch/epoch") -> Dict:
        run_config = {}
        for key, value in run.config.items():
            # Convert lists and dicts to strings
            if isinstance(value, (list, dict)):
                run_config[key] = str(value)
            else:
                run_config[key] = value

        run_history = run.history()
        run_history = run_history.replace({"Infinity": np.inf, "NaN": np.nan})

        if isinstance(considered_metrics, str):
            run_history["score"] = run_history[considered_metrics]
        elif isinstance(considered_metrics, dict):
            run_history["score"] = sum(
                run_history[metric] * weight for metric, weight in considered_metrics.items()
            )
        else:
            raise ValueError("considered_metrics must be either a string or a dictionary")
        
        best_summary = run_history.iloc[run_history["score"].argmax()]
        best_summary = {f"best:{key}": val for key, val in best_summary.items()}
        
        return {
            "run_id": run.id,
            "run_name": run.name,
            "sweep_id": run.sweep.id if run.sweep else None,
            "model": run.config.get("model"),
            **run_config,
            **{metric: run_history[metric].to_list() for metric in run_history},
            **best_summary,
            "gpu_type": run.metadata.get("gpu"),
            "cpu_count": run.metadata.get("cpu_count"),
        }

    batch_size = 16
    records = []
    futures = {}
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=batch_size)
    runs:List[Run] = api.runs("feedr/peppermint-matrix", per_page=2*batch_size-1, filters={"config.model": config["model"]})
    run_iterator = iter(runs)
    with tqdm.tqdm(total=len(runs), ncols=128) as pbar:
        while len(records) < len(runs):
            # submit new tasks if we empty slots in the batch
            while len(futures) < batch_size and len(records) + len(futures) < len(runs):
                current_runs = next(run_iterator)
                current_future = executor.submit(fetch_run_metadata, current_runs, sorting_criterion)
                futures[current_future] = current_runs

            # check for completed tasks
            finished_futures, _ = concurrent.futures.wait(futures.keys(), return_when=concurrent.futures.FIRST_COMPLETED, timeout=0.1)
            for finished_future in finished_futures:
                finished_run = futures.pop(finished_future)
                records.append(finished_future.result())
                pbar.update(1)

    # Create a Polars DataFrame from the records
    experiment_runs = pl.DataFrame(records, infer_schema_length=None)
    
# Tag run as available locally if the model files exist
local_run_ids = []
local_sweep_ids = os.listdir(f"./models/{config['model']}/")
for sweep_id in local_sweep_ids:
    local_run_ids.extend([run_id for run_id in os.listdir(f"./models/{config['model']}/{sweep_id}/")])
    
experiment_runs = experiment_runs.with_columns(
    available_locally=pl.col("run_id").is_in(local_run_ids)
)

if config["ensure_available_locally"]:
    experiment_runs = experiment_runs.filter(pl.col("available_locally") == True)

experiment_runs = experiment_runs.sort("_timestamp", descending=False)
experiment_runs = experiment_runs.with_columns(
    run_duration_second=pl.col("_runtime").list.max(),
    run_duration_minute=(pl.col("_runtime").list.max() / 60)
)
experiment_runs.select(
    pl.col("run_id"),
    pl.col("run_name"),
    pl.col("sweep_id"),
    pl.col("model"),
    pl.col("embedding_dimension"),
    pl.col("shuffle"),
    pl.col("best:epoch/epoch"),
    pl.col("best:epoch/train_loss"),
    pl.col("best:epoch/test_loss"),
    pl.col("best:epoch/test_recall@10"),
    pl.col("best:epoch/test_ndcg@10"),
)

Loading cached experiment runs from wandb/summary.parquet...
Loaded 868 runs from cache.


run_id,run_name,sweep_id,model,embedding_dimension,shuffle,best:epoch/epoch,best:epoch/train_loss,best:epoch/test_loss,best:epoch/test_recall@10,best:epoch/test_ndcg@10
str,str,str,str,i64,bool,f64,f64,f64,f64,f64
"""o94q0juk""","""logical-sweep-1""","""nbysw136""","""matrix_factorization""",256,false,52.0,0.330564,0.372714,0.026077,0.100539
"""4ftaae0p""","""stilted-sweep-3""","""nbysw136""","""matrix_factorization""",4,false,59.0,0.693148,0.693148,0.006594,0.02556
"""fway5u2z""","""breezy-sweep-4""","""nbysw136""","""matrix_factorization""",512,false,4.0,0.693147,0.693147,0.0026,0.012059
"""bphcl2xf""","""clean-sweep-2""","""nbysw136""","""matrix_factorization""",1024,false,1.0,0.237272,0.239171,0.024744,0.095121
"""fftz1dek""","""trim-sweep-5""","""nbysw136""","""matrix_factorization""",256,true,57.0,0.319744,0.366027,0.025562,0.099834
"""otb8suw9""","""scarlet-sweep-6""","""nbysw136""","""matrix_factorization""",4,true,63.0,0.133806,0.217083,0.021467,0.082659
"""lvre7srl""","""solar-sweep-7""","""nbysw136""","""matrix_factorization""",256,true,8.0,0.036125,0.194509,0.021456,0.084617
"""dcbj92eg""","""ruby-sweep-8""","""nbysw136""","""matrix_factorization""",256,false,3.0,0.078173,0.179263,0.024295,0.094158
"""x17mnyw8""","""breezy-sweep-9""","""nbysw136""","""matrix_factorization""",8,false,61.0,0.111218,0.207902,0.023699,0.089898


# Parameter Comparison

## Embedding Dimension vs Regularization | Shuffle = False

In [11]:
experiment_summary = experiment_runs.filter(pl.col("shuffle") == False).group_by("embedding_dimension", "l2_regularization").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@20").mean(),
    pl.col("best:epoch/test_ndcg@20").mean(),
).sort("embedding_dimension", "l2_regularization")
experiment_summary

embedding_dimension,l2_regularization,num_runs,best:epoch/epoch,best:epoch/test_recall@10,best:epoch/test_ndcg@10,best:epoch/test_recall@20,best:epoch/test_ndcg@20
i64,f64,u32,f64,f64,f64,f64,f64
2,0.0,4,51.25,0.015956,0.061995,0.028299,0.08201
2,1.0000e-10,4,58.5,0.015172,0.059716,0.027479,0.079558
2,1.0000e-9,4,49.25,0.014855,0.057413,0.026413,0.075915
2,1.0000e-8,5,55.6,0.015249,0.060109,0.027161,0.079271
2,0.0000001,5,55.6,0.014978,0.05851,0.026745,0.077531
2,0.000001,3,34.666667,0.014409,0.057094,0.025906,0.075099
2,0.00001,6,56.5,0.005557,0.023788,0.009413,0.030217
2,0.0001,4,9.5,0.002482,0.010624,0.004251,0.014389
2,0.001,5,11.6,0.000337,0.001539,0.000714,0.002484


In [12]:
experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(
    values=["num_runs"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/2842099446.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
2,4,4,4,5,5,3,6,4,5,6
4,3,2,4,2,2,8,4,6,4,7
8,4,9,4,3,7,4,3,4,5,2
16,6,2,2,8,2,5,3,9,4,4
32,2,3,3,5,5,6,4,6,9,8
64,3,3,2,4,8,5,3,5,5,2
128,6,2,3,4,7,4,4,3,7,3
256,4,3,4,4,5,4,4,3,5,6
512,2,2,2,5,2,3,3,7,4,4


In [13]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(
    values=["best:epoch/epoch"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/3529959983.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,51.25,58.5,49.25,55.6,55.6,34.666667,56.5,9.5,11.6,9.666667
4,37.0,54.0,48.25,50.5,60.0,57.625,59.5,7.166667,46.0,10.285714
8,53.5,45.444444,41.5,38.333333,60.857143,55.25,61.0,6.5,37.4,9.5
16,30.5,23.0,31.5,36.25,58.0,59.2,61.666667,6.222222,46.0,9.75
32,13.5,13.333333,18.666667,23.0,58.8,60.333333,60.5,5.666667,57.666667,8.625
64,8.333333,9.0,8.5,16.0,59.625,53.2,62.0,5.4,60.6,8.5
128,7.666667,3.0,3.333333,5.0,55.714286,58.5,62.0,5.333333,57.714286,8.666667
256,5.25,4.666667,3.25,3.75,55.6,49.25,61.5,5.333333,57.6,8.5
512,1.0,1.0,1.0,2.0,31.0,53.0,62.666667,5.571429,62.25,8.5


In [14]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(
    values=["best:epoch/test_recall@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/128151351.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.028299,0.027479,0.026413,0.027161,0.026745,0.025906,0.009413,0.004251,0.000714,0.00117
4,0.035486,0.036021,0.034871,0.035595,0.036566,0.036103,0.010423,0.004288,0.00062,0.001387
8,0.03909,0.039219,0.039121,0.040172,0.042089,0.041671,0.011141,0.004165,0.000604,0.001154
16,0.040085,0.040318,0.04048,0.041284,0.045753,0.04373,0.011708,0.004219,0.000635,0.001355
32,0.040401,0.041019,0.040565,0.041324,0.047913,0.044841,0.011938,0.004357,0.000637,0.001499
64,0.039906,0.041457,0.041391,0.041036,0.048462,0.045193,0.012688,0.004215,0.000671,0.001687
128,0.03902,0.042331,0.041844,0.042047,0.048909,0.0452,0.012909,0.004322,0.000708,0.001859
256,0.038145,0.041366,0.04183,0.04228,0.047799,0.045084,0.013638,0.004303,0.000725,0.001742
512,0.040658,0.04213,0.041295,0.041465,0.044598,0.04552,0.013945,0.004337,0.000747,0.001457


In [15]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(
    values=["best:epoch/test_recall@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/128151351.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.028299,0.027479,0.026413,0.027161,0.026745,0.025906,0.009413,0.004251,0.000714,0.00117
4,0.035486,0.036021,0.034871,0.035595,0.036566,0.036103,0.010423,0.004288,0.00062,0.001387
8,0.03909,0.039219,0.039121,0.040172,0.042089,0.041671,0.011141,0.004165,0.000604,0.001154
16,0.040085,0.040318,0.04048,0.041284,0.045753,0.04373,0.011708,0.004219,0.000635,0.001355
32,0.040401,0.041019,0.040565,0.041324,0.047913,0.044841,0.011938,0.004357,0.000637,0.001499
64,0.039906,0.041457,0.041391,0.041036,0.048462,0.045193,0.012688,0.004215,0.000671,0.001687
128,0.03902,0.042331,0.041844,0.042047,0.048909,0.0452,0.012909,0.004322,0.000708,0.001859
256,0.038145,0.041366,0.04183,0.04228,0.047799,0.045084,0.013638,0.004303,0.000725,0.001742
512,0.040658,0.04213,0.041295,0.041465,0.044598,0.04552,0.013945,0.004337,0.000747,0.001457


In [16]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@20"]].pivot(
    values=["best:epoch/test_ndcg@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/3686121945.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@20"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.08201,0.079558,0.075915,0.079271,0.077531,0.075099,0.030217,0.014389,0.002484,0.003616
4,0.101947,0.1025,0.098071,0.100571,0.102822,0.102952,0.033224,0.015206,0.002254,0.004341
8,0.110391,0.110855,0.110785,0.112272,0.11766,0.117419,0.033542,0.014861,0.002137,0.003596
16,0.113333,0.114327,0.114361,0.116159,0.128185,0.122634,0.036012,0.014989,0.002359,0.003994
32,0.113685,0.115382,0.114562,0.11644,0.132597,0.125658,0.037479,0.015193,0.00237,0.004821
64,0.113178,0.116776,0.115964,0.116176,0.13426,0.12657,0.038883,0.014786,0.002622,0.005648
128,0.111663,0.119628,0.118013,0.118536,0.134724,0.126154,0.040305,0.015177,0.002728,0.006084
256,0.108346,0.116493,0.11857,0.119785,0.132158,0.126971,0.041902,0.015182,0.002757,0.00487
512,0.113271,0.11756,0.114751,0.116177,0.125115,0.126861,0.042879,0.015423,0.002837,0.004465


## Embedding Dimension vs Regularization | Shuffle = True

In [17]:
experiment_summary = experiment_runs.filter(pl.col("shuffle") == True).group_by("embedding_dimension", "l2_regularization").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@20").mean(),
    pl.col("best:epoch/test_ndcg@20").mean(),
).sort("embedding_dimension", "l2_regularization")

In [18]:
experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(
    values=["num_runs"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/2842099446.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
2,5,3,1,5,2,3,5,2,7,4
4,5,3,5,6,5,4,5,8,5,3
8,4,4,2,5,2,6,5,5,4,7
16,2,3,2,10,4,4,9,4,5,6
32,5,2,2,3,5,10,3,3,3,7
64,3,5,4,4,6,5,3,3,4,7
128,4,3,4,5,10,4,6,5,4,4
256,4,2,2,5,5,5,3,4,3,5
512,3,3,4,5,5,7,2,2,5,7


In [19]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(
    values=["best:epoch/epoch"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/3529959983.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,47.4,48.666667,61.0,55.6,53.0,50.666667,45.6,11.5,58.571429,9.0
4,58.2,60.666667,48.6,55.5,61.8,60.0,43.2,8.75,61.4,8.333333
8,48.0,53.0,62.5,55.2,60.0,59.166667,44.0,10.2,59.25,8.857143
16,48.5,33.333333,52.5,52.3,60.75,56.0,45.444444,11.0,63.0,8.5
32,18.0,28.0,21.5,41.0,59.6,54.7,48.0,11.666667,61.666667,9.0
64,16.0,13.8,15.5,21.0,58.5,52.2,53.0,12.0,62.5,8.0
128,15.5,12.333333,13.75,14.8,57.8,53.25,58.5,10.6,61.75,8.75
256,11.0,11.0,10.0,12.8,53.8,54.6,0.0,9.75,61.666667,7.8
512,6.666667,2.333333,9.75,10.0,38.0,49.571429,0.0,10.5,60.0,8.0


In [20]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(
    values=["best:epoch/test_recall@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/128151351.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.026802,0.027117,0.024519,0.027582,0.027732,0.026763,0.022443,0.004749,0.001253,0.001291
4,0.03502,0.035082,0.035621,0.035248,0.036723,0.035059,0.023605,0.004646,0.001419,0.001035
8,0.039135,0.039344,0.039087,0.04002,0.042581,0.041639,0.023367,0.004607,0.001258,0.001233
16,0.039995,0.039796,0.039979,0.041158,0.0465,0.04392,0.023699,0.004754,0.001472,0.001397
32,0.039954,0.039639,0.04,0.040843,0.047524,0.044698,0.02361,0.004856,0.001429,0.001452
64,0.039525,0.039531,0.039444,0.040625,0.048557,0.045053,0.024785,0.004896,0.001584,0.001608
128,0.038355,0.038835,0.038436,0.039411,0.048694,0.044757,0.021124,0.004801,0.001645,0.001317
256,0.037206,0.03723,0.038289,0.038295,0.047734,0.045243,0.017743,0.004785,0.001716,0.001571
512,0.035717,0.036799,0.036013,0.036662,0.044243,0.044985,0.017599,0.004849,0.001893,0.001534


In [21]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@20"]].pivot(
    values=["best:epoch/test_ndcg@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_2467735/3686121945.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@20"]].pivot(


embedding_dimension,0.0,1e-10,1e-9,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.077845,0.080488,0.073735,0.080168,0.080954,0.078462,0.066975,0.016989,0.005061,0.004804
4,0.099924,0.099181,0.101943,0.100334,0.103918,0.101334,0.070803,0.016738,0.005597,0.003315
8,0.110453,0.110979,0.109776,0.113028,0.119901,0.116816,0.071034,0.016199,0.005,0.004304
16,0.113371,0.112979,0.112719,0.115682,0.128256,0.122477,0.071606,0.017082,0.005616,0.004364
32,0.113714,0.112975,0.113315,0.114622,0.131433,0.125395,0.071054,0.016999,0.005687,0.004393
64,0.112327,0.112011,0.112694,0.114441,0.133791,0.126345,0.074572,0.016865,0.006101,0.004945
128,0.109684,0.110493,0.109764,0.112396,0.134253,0.125869,0.065612,0.017077,0.006286,0.004135
256,0.106902,0.107047,0.108992,0.10984,0.131401,0.125951,0.05364,0.017016,0.006449,0.004893
512,0.103055,0.106343,0.104065,0.105173,0.124356,0.126202,0.052732,0.017076,0.007156,0.004955


# Cross-GPU Training

In [15]:
experiment_runs.group_by("embedding_dimension").agg(
    pl.col("run_duration_minute").mean()
).sort("embedding_dimension")

embedding_dimension,run_duration_minute
i64,f64
2,22.742412
4,22.665075
8,23.885482
16,22.985601
32,23.285895
64,21.641968
128,23.73886
256,24.837704
512,25.84378


In [16]:
experiment_runs.group_by("gpu_type").agg(
    pl.col("run_duration_minute").mean()
).sort("gpu_type")

gpu_type,run_duration_minute
str,f64
"""NVIDIA A100-SXM4-40GB""",47.189427
"""NVIDIA A10G""",23.009375
"""NVIDIA L4""",28.125109


failed to send, dropping 2 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 6 additional messages skipped


In [17]:
experiment_summary = experiment_runs.filter(pl.col("l2_regularization") == 0.).group_by("embedding_dimension", "gpu_type").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@50").mean(),
    pl.col("best:epoch/test_ndcg@50").mean(),
).sort("embedding_dimension", "gpu_type")
experiment_summary

embedding_dimension,gpu_type,num_runs,best:epoch/epoch,best:epoch/test_recall@10,best:epoch/test_ndcg@10,best:epoch/test_recall@50,best:epoch/test_ndcg@50
i64,str,u32,f64,f64,f64,f64,f64
2,"""NVIDIA A100-SXM4-40GB""",1,63.0,0.014696,0.059235,0.05268,0.105463
2,"""NVIDIA A10G""",19,63.0,0.014549,0.056882,0.053937,0.103935
2,"""NVIDIA L4""",1,63.0,0.014849,0.058681,0.056454,0.10793
4,"""NVIDIA A100-SXM4-40GB""",1,63.0,0.019013,0.074248,0.070179,0.129829
4,"""NVIDIA A10G""",14,63.0,0.019431,0.075468,0.069966,0.130528
4,"""NVIDIA L4""",4,63.0,0.019367,0.075652,0.069782,0.130409
8,"""NVIDIA A100-SXM4-40GB""",2,63.0,0.02136,0.082498,0.078069,0.141937
8,"""NVIDIA A10G""",19,63.0,0.021914,0.084904,0.078703,0.143965
8,"""NVIDIA L4""",3,63.0,0.021715,0.083619,0.078856,0.143085


In [18]:
experiment_summary[["embedding_dimension", "gpu_type", "num_runs"]].pivot(
    values=["num_runs"],
    index="embedding_dimension",
    columns="gpu_type"
)

/tmp/ipykernel_900224/1667364229.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "gpu_type", "num_runs"]].pivot(


embedding_dimension,NVIDIA A100-SXM4-40GB,NVIDIA A10G,NVIDIA L4
i64,u32,u32,u32
2,1,19,1
4,1,14,4
8,2,19,3
16,1,27,1
32,null,27,3
64,1,21,2
128,null,26,null
256,3,23,4
512,2,20,3


In [19]:
experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_recall@50"]].pivot(
    values=["best:epoch/test_recall@50"],
    index="embedding_dimension",
    columns="gpu_type"
)

/tmp/ipykernel_900224/93591780.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_recall@50"]].pivot(


embedding_dimension,NVIDIA A100-SXM4-40GB,NVIDIA A10G,NVIDIA L4
i64,f64,f64,f64
2,0.05268,0.053937,0.056454
4,0.070179,0.069966,0.069782
8,0.078069,0.078703,0.078856
16,0.079871,0.079915,0.081216
32,null,0.077398,0.077563
64,0.07268,0.073611,0.072878
128,null,0.069959,null
256,0.066112,0.066402,0.066835
512,0.062456,0.063286,0.063337


In [20]:
experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_ndcg@50"]].pivot(
    values=["best:epoch/test_ndcg@50"],
    index="embedding_dimension",
    columns="gpu_type"
)

/tmp/ipykernel_900224/2601597168.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_ndcg@50"]].pivot(


embedding_dimension,NVIDIA A100-SXM4-40GB,NVIDIA A10G,NVIDIA L4
i64,f64,f64,f64
2,0.105463,0.103935,0.10793
4,0.129829,0.130528,0.130409
8,0.141937,0.143965,0.143085
16,0.146282,0.145748,0.147393
32,null,0.142054,0.142608
64,0.136999,0.136925,0.135871
128,null,0.132737,null
256,0.127316,0.128326,0.128494
512,0.123614,0.12421,0.124616
